# Importations

In [2]:
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from tqdm import tqdm
import neuroseries as nts
import scipy as sp
import seaborn as sns

import bk.load
import bk.compute
import bk.plot
import bk.signal


%matplotlib qt

# Variables

In [59]:
bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= 'Rat08/Rat08-20130709')
#load the path name, session name, rat number, day number, and the number of channels in the session 

Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709


True

In [60]:
states = bk.load.states() 
#load the time intervals of the states of the animal (Rem, sws, wake, drowsy) in a dict

In [61]:
neurons,metadata = bk.load.spikes() 
#load the metadata (pd dataframe where informations about the neurons are: 
#rat, day, shank, id on the shank, region, and type)

#load the neurons (np array containing nt frames from each neuron, 
#in each nt frame there is the timing of the spike (index) and values are nan)

Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


In [62]:
neurons_pyr = neurons[metadata['Type'] == 'Pyr'] #only pyramidal neurons
neurons_pyr_hpc = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'Hpc')] #only hippocampal pyramidal 
neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')] #only bla pyramidal

neurons_int = neurons[metadata['Type'] == 'Int'] #only interneurons
neurons_int_hpc = neurons[(metadata['Type'] == 'Int') & (metadata['Region'] == 'Hpc')] #only hippocampal interneurons 
neurons_int_bla = neurons[(metadata['Type'] == 'Int') & (metadata['Region'] == 'BLA')] #only bla interneurons

neurons_pyr_ordered = np.concatenate((neurons_pyr_hpc, neurons_pyr_bla)) #all pyramidal neurons, but in order
neurons_int_ordered = np.concatenate((neurons_int_hpc, neurons_int_bla)) #all interneurons, but in order

In [64]:
if len(neurons_pyr_bla) == 0 or len(neurons_int_bla) == 0:
    print('Careful, there are not enough neurons')

In [65]:
window = 1 #the size of the time bins used to make the correlation matrix, in sec
smallbins = 0.1 #the size of the small bins used to compute binspikes
shift = 1 #the shift from one bin to another, in sec 

# Functions 

In [8]:
def mean_synchrony(neurons, asymetry=False, neurons2=False, window=1, smallbins=0.1, shift=1):
    '''
    MC 29/10/21
    This function computes the mean synchrony for neurons across time
    Inputs: 
        neurons: the neurons you want to compute the synchrony on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of synchrony among the neurons (in sec), default = 1
        shift: the shift between one window and the following (in sec), default = 1
        asymetry: if you want to compute this synchrony between 2 different populations of neurons, default = False
        neurons2: the second population of neurons, default = False
    Outputs:
        a neuroseries data frame with the mean of synchrony across time
    '''
    if asymetry: #if we want to compute the synchrony between 2 different populations of neurons
        neurons = np.concatenate((neurons, neurons2)) # concatenate the 2 populations of neurons
        l_neurons2 = len(neurons2) #take the length of the second population, it will be used to extract
                                   #only the part of the corrcoef matrix between the 2 populations and not between
                                   #themselves
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_sync = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean synchrony across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        corrcoef = np.corrcoef(binned_neurons[:,int(j-b/2):int(j + b/2)]) #compute the corrcoef in a [-b/2:b/2] window
        if asymetry: 
            corrcoef = corrcoef[l_neurons2:,:l_neurons2] #extract only the corrcoef matrix between the 2 populations
        mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
    t=t[::c] #extract t timings at c steps
    toreturn = nts.Tsd(t[:-1], mean_sync, time_units='s') #create a neuroseries frame with the means of synchrony
    return toreturn

In [9]:
def mean_synchrony_nan(neurons, asymetry=False, neurons2=False, window=2, smallbins=0.1, shift=1):
    '''
    MC 02/11/21
    This function computes the mean synchrony for neurons across time, but with NaN values changed to 0
    Inputs: 
        neurons: the neurons you want to compute the synchrony on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of synchrony among the neurons (in sec), default = 2
        shift: the shift between one window and the following (in sec), default = 1
        asymetry: if you want to compute this synchrony between 2 different populations of neurons, default = False
        neurons2: the second population of neurons, default = False
    Outputs:
        a neuroseries data frame with the mean of synchrony across time
    '''
    if asymetry: #if we want to compute the synchrony between 2 different populations of neurons
        neurons = np.concatenate((neurons, neurons2)) # concatenate the 2 populations of neurons
        l_neurons2 = len(neurons2) #take the length of the second population, it will be used to extract
                                   #only the part of the corrcoef matrix between the 2 populations and not between
                                   #themselves
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_sync = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean synchrony across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        corrcoef = np.corrcoef(binned_neurons[:,int(j-b/2):int(j + b/2)]) #compute the corrcoef in a [-b/2:b/2] window
        if asymetry: 
            corrcoef = corrcoef[l_neurons2:,:l_neurons2] #extract only the corrcoef matrix between the 2 populations
        corrcoef[np.isnan(corrcoef)] = 0 #all nan values are changed to 0
        mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
    t=t[::c] #extract t timings at c steps
    toreturn = nts.Tsd(t[:-1], mean_sync, time_units='s') #create a neuroseries frame with the means of synchrony
    return toreturn

In [10]:
def mean_firing_rate(neurons, window=2, smallbins=0.1, shift=1):
    '''
    MC 02/11/21
    This function computes the mean firing rate of a neuron population across time
    Inputs: 
        neurons: the neurons you want to compute the mean firing rate on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of firing rate among the neurons (in sec), default = 2
        shift: the shift between one window and the following (in sec), default = 1
    Outputs:
        a neuroseries data frame with the mean of firing rate across time
    '''
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_fr = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean firing rate across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        mean_fr[i] = np.nanmean(binned_neurons[:,int(j-b/2):int(j + b/2)])/window #compute the mean of firing rate for each window, 
                                                                                  #disregarding the nan
    t=t[::c] #extract t timings at c steps
    toreturn = nts.Tsd(t[:-1], mean_fr, time_units='s') #create a neuroseries frame with the means of firing rate
    return toreturn

In [11]:
def pourcent_active_neurons(neurons, window=1):
    '''
    MC 03/11/21
    This function computes the pourcentage of active neurons across time
    Inputs: 
        neurons: the neurons you want to compute the pourcentages on
        window: the size of the time window in which to compute the pourcentage of active neurons (in sec), default = 1
    Outputs:
        a neuroseries data frame with the pourcentage of active neurons across time
    '''
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=window) #count the number of spikes for each neuron in windows
    pourcent_active = np.zeros(int(binned_neurons.shape[1])) #initialisation of the pourcentages across time 
    for i,j in enumerate(tqdm(range(0, int(binned_neurons.shape[1]), window))): #for each window
        pourcent_active[i] = np.count_nonzero(binned_neurons[:,j])/len(neurons) #compute the pourcentage for each window
    t=t[::window] #extract t timings at shift steps
    toreturn = nts.Tsd(t, pourcent_active, time_units='s') #create a neuroseries frame with the pourcentages
    return toreturn

In [12]:
def normalization(neurons):
    '''
    MC 08/11/21
    this function normalizes the pourcentage of active cells to the mean firing rate
    Input : the neurons you want to compute this normalization on
    Output : a neuroseries timeframe of this normalized pourcentages of active cells 
    '''
    t,_ = bk.compute.binSpikes(neurons, binSize=1)
    pac = pourcent_active_neurons(neurons, window = 1) #compute the pourcent of active cells across time
    pac = bk.compute.nts_smooth(pac, 100, 50) #smoothing the results 
    mfr = mean_firing_rate(neurons, window=2, shift=1) #compute the mean FR across time 
    mfr = bk.compute.nts_smooth(mfr, 100, 50) #smoothing the results
    toreturn = [(i/j) for i,j in zip(pac.values, mfr.values)] #normalize the pac to the mfr
    toreturn = nts.Tsd(t, toreturn, time_units = 's') #create the tsd
    return toreturn
    

In [13]:
def synchrony_around_transitions(transitions, mean_sync):
    '''
    MC 13/12/21
    This function computes the mean of synchrony around transitions REM/sws or sws/REM, 
    and the mean of synchrony of the whole epochs considered (REM or sws)
    Inputs: 
        transitions: the timing of the transitions
        mean_sync: the mean synchrony across the session
    Output: a dataframe with the different means for each type 
    '''
    
    trans = {'Rem before transition': [], 
            'Rem after transition': [],
             'Rem': [],
            'sws before transition' : [], 
            'sws after transition': [],
            'sws': []}
    
    transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
    transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values
    
    for i  in transitions_rem_sws:
        interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='us')
        trans['Rem before transition'].append(np.mean(mean_sync.restrict(interval_rem).values))
        interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='us')
        trans['sws after transition'].append(np.mean(mean_sync.restrict(interval_sws).values))

    for i  in transitions_sws_rem:
        interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='us')
        trans['sws before transition'].append(np.mean(mean_sync.restrict(interval_sws).values))
        interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='us')
        trans['Rem after transition'].append(np.mean(mean_sync.restrict(interval_rem).values))

    for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
        interval = nts.IntervalSet(i,j, time_units='s')
        trans['Rem'].append(np.mean(mean_sync.restrict(interval).values))

    for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
        interval = nts.IntervalSet(i,j, time_units='s')
        trans['sws'].append(np.mean(mean_sync.restrict(interval).values))

    trans = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in trans.items() ]))
    return trans

# Computation of variables

In [66]:
mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
mean_sync_int_bla = mean_synchrony(neurons_int_bla, window=1, smallbins=0.1, shift=1)

mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
mean_sync_int_bla_s = bk.compute.nts_smooth(mean_sync_int_bla, 100,50)

  0%|                                                                                        | 0/25510 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 21%|███████████████▋                                                           | 5345/25510 [00:03<00:14, 1433.94it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:09<00:00, 2600.95it/s]


In [67]:
mean_sync_nan_pyr_bla = mean_synchrony_nan(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
mean_sync_nan_int_bla = mean_synchrony_nan(neurons_int_bla, window=1, smallbins=0.1, shift=1)

mean_sync_nan_pyr_bla_s = bk.compute.nts_smooth(mean_sync_nan_pyr_bla, 100,50)
mean_sync_nan_int_bla_s = bk.compute.nts_smooth(mean_sync_nan_int_bla, 100,50)

100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:09<00:00, 2572.78it/s]


In [68]:
m_asym_bla = mean_synchrony(neurons_pyr_bla, asymetry=True, 
                            neurons2=neurons_int_bla, window=1, 
                            smallbins = 0.1, shift= 1)

m_asym_bla_s = bk.compute.nts_smooth(m_asym_bla, 30, 30)

  0%|                                                                                        | 0/25510 [00:00<?, ?it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|███████████████████████████████████████████████████████████████████████████| 25510/25510 [00:26<00:00, 974.15it/s]


In [69]:
mean_fr_pyr_bla = mean_firing_rate(neurons_pyr_bla)
mean_fr_int_bla = mean_firing_rate(neurons_int_bla)

mean_fr_pyr_bla_s = bk.compute.nts_smooth(mean_fr_pyr_bla, 100,50)
mean_fr_int_bla_s = bk.compute.nts_smooth(mean_fr_int_bla, 100,50)

100%|█████████████████████████████████████████████████████████████████████████| 25509/25509 [00:01<00:00, 24848.63it/s]


In [70]:
pourcent_active_pyr_bla = pourcent_active_neurons(neurons_pyr_bla)
pourcent_active_int_bla = pourcent_active_neurons(neurons_int_bla)

pourcent_active_pyr_bla_s = bk.compute.nts_smooth(pourcent_active_pyr_bla, 100,50)
pourcent_active_int_bla_s = bk.compute.nts_smooth(pourcent_active_int_bla, 100,50)

100%|████████████████████████████████████████████████████████████████████████| 25510/25510 [00:00<00:00, 176399.48it/s]


In [71]:
transitions = bk.compute.transitions_times(states)

transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

# Matrices de corrélation et calculs de moyenne de synchronie

### Asymetrical matrices BLA/HPC with pyr and int

In [45]:
m_asym_pyr = mean_synchrony(neurons_pyr_bla, asymetry=True, neurons2=neurons_pyr_hpc, window=1, smallbins = 0.025, shift= 1)
m_asym_int = mean_synchrony(neurons_int_bla, asymetry=True, neurons2=neurons_int_hpc, window=1, smallbins = 0.025, shift= 1)
m_asym_bla = mean_synchrony(neurons_pyr_bla, asymetry=True, neurons2=neurons_int_bla, window=1, smallbins = 0.025, shift= 1)
m_asym_hpc = mean_synchrony(neurons_pyr_hpc, asymetry=True, neurons2=neurons_int_hpc, window=1, smallbins = 0.025, shift= 1)

  0%|                                                                                        | 0/22639 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
  4%|██▋                                                                          | 808/22639 [00:00<00:22, 987.60it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_23364/4247394187.py:27: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef)
100%|██████████████████████████████████████████████████████████████████████████| 22639/22639 [00:19<00:00, 1162.21it/s]


In [46]:
m_asym_pyr_s = bk.compute.nts_smooth(m_asym_pyr, 10,5)
m_asym_int_s = bk.compute.nts_smooth(m_asym_int, 10,5)
m_asym_bla_s = bk.compute.nts_smooth(m_asym_bla, 10,5)
m_asym_hpc_s = bk.compute.nts_smooth(m_asym_hpc, 10,5)

In [65]:
#for asymetrical matrices (between hpc and bla)
#To get the correlation coefficient between hpc and bla, I concatenate the 2, then I compute the matrix 
#but I only take the part where it is between the two, and not between themselves. Then I can compute the mean

l_pyr_bla = len(neurons_pyr_bla)
l_pyr_hpc = len(neurons_pyr_hpc)
l_int_bla = len(neurons_int_bla)
l_int_hpc = len(neurons_int_hpc)
#compute their length to only take the part of the matrix where it is asymetrical

mean_corr_pyr = []
mean_corr_int = []

for i in interval:
    corrcoef_pyr = np.corrcoef(binned_pyr[:,int(i):int(i + bins)]) 
    corrcoef_pyr = corrcoef_pyr[l_pyr_hpc:-1, l_pyr_bla:-1]
    mean_corr_pyr.append(np.nanmean(corrcoef_pyr))
    corrcoef_int = np.corrcoef(binned_int[:,int(i):int(i + bins)])  
    corrcoef_int = corrcoef_int[l_int_hpc:-1, l_int_bla:-1]
    mean_corr_int.append(np.nanmean(corrcoef_int))


C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


### Plotting

In [48]:
plt.figure()
plt.subplot(4,1,1)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_bla[15000:].as_units('s'), label='pyr bla', color = 'g')
plt.plot(m_pyr_hpc[15000:].as_units('s'), label='pyr hpc', color = 'b')
plt.plot(m_asym_pyr[15000:].as_units('s'), label ='synchro pyr BLA/HPC', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,2)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_int_bla[15000:].as_units('s'), label='int bla', color = 'g', linestyle='--')
plt.plot(m_int_hpc[15000:].as_units('s'), label='int hpc', color = 'b', linestyle='--')
plt.plot(m_asym_int[15000:].as_units('s'), label ='synchro int BLA/HPC', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,3)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_bla[15000:].as_units('s'), label='pyr bla', color = 'g')
plt.plot(m_int_bla[15000:].as_units('s'), label='int bla', color = 'g', linestyle='--')
plt.plot(m_asym_bla[15000:].as_units('s'), label ='synchro BLA pyr/int', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,4)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_hpc[15000:].as_units('s'), label='pyr hpc', color = 'b')
plt.plot(m_int_hpc[15000:].as_units('s'), label='int hpc', color = 'b', linestyle='--')
plt.plot(m_asym_hpc[15000:].as_units('s'), label ='synchro HPC Pyr/int', color='r')
plt.xlim(15000,22000)
plt.legend()

In [23]:
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_fr_pyr_bla_s.as_units('s'), label='mean firing rate', color = 'g')
plt.plot(pourcent_active_pyr_bla_s.as_units('s'), label='pourcent active', color = 'b')
plt.plot(mean_sync_pyr_bla_s.as_units('s'), label='mean synchrony', color = 'r')

plt.legend()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_pyr_bla_s.restrict(states['Rem']).values, 
                mean_sync_pyr_bla_s.restrict(states['Rem']).values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_pyr_bla_s.restrict(states['sws']).values, 
            mean_sync_pyr_bla_s.restrict(states['sws']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
sns.regplot(mean_fr_pyr_bla_s[100:-100].restrict(states['wake']).values, 
            mean_sync_pyr_bla_s[100:-100].restrict(states['wake']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA pyramidal cells')
g.legend(labels=['Rem', 'sws', 'Wake'])

plt.subplot(1,2,2)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, 
                mean_sync_nan_int_bla_s.restrict(states['Rem']).values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, 
            mean_sync_nan_int_bla_s.restrict(states['sws']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, 
            mean_sync_nan_int_bla_s[100:-100].restrict(states['wake']).values, 
            scatter_kws={'alpha':0.1, 's':5}, robust=True, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'pourcent of active cells',
      title='Correlation between pourcent of active cells and firing rate in BLA pyramidal cells')
g.legend(labels=['Rem', 'sws', 'Wake'])

In [88]:
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_pyr_bla_s[100:-100].values, mean_sync_pyr_bla_s[100:-100].values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA pyramidal')

plt.subplot(1,2,2)
g = sns.regplot(mean_fr_pyr_bla_s[100:-100].values, pourcent_active_pyr_bla_s[100:-100].values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
g.set(xlabel ='mean firing rate', ylabel = 'mean pourcent active',
      title='Correlation between pourcent of active cells and firing rate in BLA pyramidal')

NameError: name 'mean_fr_pyr_bla_s' is not defined

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pourcent_active_pyr_bla_s[100:-100].values,
           mean_sync_pyr_bla_s[100:-100].values,
           mean_fr_pyr_bla_s[100:-100].values)

In [34]:
#plot des moyennes de synchronie en fonction des states 
plt.subplot(2,1,1)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_pyr_bla_s.as_units('s'), color='g',linewidth=2, label='pyr BLA')
plt.plot(mean_pyr_hpc_s.as_units('s'), color='b',linewidth=2, label='pyr HPC')
plt.legend()

plt.subplot(2,1,2)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_int_bla_s.as_units('s'), color='g',linewidth=2, label='int BLA')
plt.plot(mean_int_hpc_s.as_units('s'), color='b',linewidth=2, label='int HPC')
plt.legend()

In [102]:
plt.figure(figsize=(12,6))

bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_sync_pyr_bla_s.as_units('s'), color='g',linewidth=2, label='synchrony BLA')
plt.plot(mean_sync_pyr_hpc_s.as_units('s'), color='r',linewidth=2, label='synchrony Hpc')
plt.legend()
plt.savefig('C:/Users/maell/Documents/ENS/Cours/M2/S1/TINS/Figure/synchrony_across_time.svg')

# Tests 

In [81]:
for i in os.listdir('E:/DATA/GG-Dataset/Rat08/'):
    print(i)

Rat08-20130713
Rat08-20130708
Rat08-20130709
Rat08-20130710
Rat08-20130711
Rat08-20130712
Rat08-20130715
Rat08-20130716
Rat08-20130717
Rat08-20130718
Rat08-20130719
Rat08-20130720
Rat08-20130722
ShankChannels.mat


In [89]:
means =[]
fig, ax = plt.subplots()
for i in os.listdir('E:/DATA/GG-Dataset/Rat08/'):
    if i.endswith('.mat') or i == 'Rat08-20130720':
        continue
        
    local_path= f'Rat08/{i}'
    bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
    states = bk.load.states() 
    neurons,metadata = bk.load.spikes() 
    
    neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
    if len(neurons_pyr_bla) < 10: 
        continue
    
    mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
    mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
    
    transitions = bk.compute.transitions_times(states)
    transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
    transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

    for i  in transitions_rem_sws:
        interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
        m = mean_sync_pyr_bla.restrict(interval).values
        means.append(m)
        plt.plot(m, color = 'grey', alpha = 0.3)
a = mpatches.Rectangle((0,0), 20, 0.3, facecolor = 'orange', alpha = 0.5)
ax.add_patch(a)
mean = np.mean(means, axis=0)
plt.plot(mean, color = 'r', linewidth=4)
plt.title(f'transitions rem to sws, rat 08, session: {local_path}')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:17<00:00, 1299.95it/s]


Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy


  0%|                                                                                        | 0/24758 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 24758/24758 [00:09<00:00, 2509.79it/s]


Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


...

  0%|                                                                                        | 0/25112 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 15%|███████████▎                                                               | 3791/25112 [00:01<00:09, 2337.12it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25112/25112 [00:12<00:00, 2052.66it/s]


Rat : 8 on day : 9
Working with session Rat08-20130717 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130717
Data already saved in Numpy format, loading them from here:
Rat08-20130717-neurons.npy
Rat08-20130717-metadata.npy


  0%|                                                                                        | 0/23243 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 23243/23243 [00:11<00:00, 1995.99it/s]


Rat : 8 on day : 10
Working with session Rat08-20130718 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130718
Data already saved in Numpy format, loading them from here:
Rat08-20130718-neurons.npy
Rat08-20130718-metadata.npy
Rat : 8 on day : 11
Working with session Rat08-20130719 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130719
Data already saved in Numpy format, loading them from here:
Rat08-20130719-neurons.npy
Rat08-20130719-metadata.npy
Rat : 8 on day : 13
Working with session Rat08-20130722 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130722


FileNotFoundError: [Errno 2] No such file or directory: 'E:/DATA/GG-Dataset/Rat08/Rat08-20130722/States.mat'

In [91]:
test = {'Rem BLA': [], 'sws BLA': [], 'Rem Hpc' : [], 'sws Hpc': []}
for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    test['Rem BLA'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    test['Rem Hpc'].append(np.mean(mean_sync_pyr_hpc.restrict(interval).values))
for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    test['sws BLA'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    test['sws Hpc'].append(np.mean(mean_sync_pyr_hpc.restrict(interval).values))

In [41]:
sp.stats.kruskal(t['sws before transition'], t['sws'], nan_policy='omit')

KruskalResult(statistic=3.3295318127250937, pvalue=0.0680462420014518)

In [23]:
t = synchrony_around_transitions(transitions, mean_sync_pyr_bla_s)

In [21]:
sns.boxplot(data = t, whis=np.inf)
sns.swarmplot(data = t, color=".2")
#plt.savefig('C:/Users/maell/Documents/ENS/Cours/M2/S1/TINS/Figure/means_synchrony_rem_sws.svg')

NameError: name 't' is not defined

In [57]:
len(neurons_pyr_bla)

15

In [50]:
def plot_synchrony_over_time_transitions(transitions, mean_sync, rem_to_sws=True):
    '''
    MC 15/12/21
    This function draws the synchrony over time around the transitions 
    Inputs:
        transitions: 
        mean_sync: the mean of synchrony over time 
        rem_to_sws: if we look at rem to sws transitions or sws to rem transitions (by default: rem_to_sws)
    Output:
    A figure with the synchrony over time drawn
    '''
    means =[]
    fig, ax = plt.subplots()
    for i  in transitions_rem_sws:
        interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
        m = mean_sync_pyr_bla.restrict(interval).values
        means.append(m)
        plt.plot(m, color = 'grey', alpha = 0.3)
    a = mpatches.Rectangle((0,0), 20, 0.3, facecolor = 'orange', alpha = 0.5)
    ax.add_patch(a)
    mean = np.mean(means, axis=0)
    plt.plot(mean, color = 'r', linewidth=4)

In [51]:
means =[]
fig, ax = plt.subplots()
for i  in transitions_sws_rem:
    interval = nts.IntervalSet(i-150_000_000, i+ 50_000_000, time_units='us')
    m = mean_sync_pyr_bla.restrict(interval).values
    means.append(m)
    plt.plot(m, color = 'grey', alpha = 0.3)
a = mpatches.Rectangle((150,0), 50, 0.3, facecolor = 'orange', alpha = 0.5)
ax.add_patch(a)
mean = np.mean(means, axis=0)
plt.plot(mean, color = 'r', linewidth=4)

In [156]:
trans = {'Rem before transition': [], 
        'Rem after transition': [],
         'Rem': [],
        'sws before transition' : [], 
        'sws after transition': [],
        'sws': []}
for i  in transitions_rem_sws:
    interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='s')
    trans['Rem before transition'].append(np.mean(mean_sync_pyr_bla.restrict(interval_rem).values))
    interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='s')
    trans['sws after transition'].append(np.mean(mean_sync_pyr_hpc.restrict(interval_sws).values))
    
for i  in transitions_sws_rem:
    interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='s')
    trans['sws before transition'].append(np.mean(mean_sync_pyr_bla.restrict(interval_sws).values))
    interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='s')
    trans['Rem after transition'].append(np.mean(mean_sync_pyr_hpc.restrict(interval_rem).values))
    
for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    trans['Rem'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    
for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    trans['sws'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))

C:\Users\maell\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\maell\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Correlation between firing rate and synchrony

In [66]:
plt.figure()
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, mean_sync_int_bla_s.restrict(states['Rem']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, mean_sync_int_bla_s.restrict(states['sws']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, mean_sync_int_bla_s[100:-100].restrict(states['wake']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA interneurons')
g.legend(labels=['Rem', 'sws', 'Wake'])
plt.subplot(1,2,2)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, mean_sync_nan_int_bla_s.restrict(states['Rem']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, mean_sync_nan_int_bla_s.restrict(states['sws']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, mean_sync_nan_int_bla_s[100:-100].restrict(states['wake']).values, scatter_kws={'alpha':0.1, 's':5}, robust=True, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony (with nan to 0)',
      title='Correlation between synchrony (with nan to 0) and firing rate in BLA interneurons')
g.legend(labels=['Rem', 'sws', 'Wake'])

C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anacond

In [96]:
plt.subplot(2,2,1)
plt.plot(mean_fr_pyr_hpc_s[100:-100].as_units('s'), mean_sync_pyr_hpc_s[100:-100].as_units('s'), '.')
plt.title('pyr hpc, Pearson coeff = '+str(corr_pyr_hpc[1,0]))
plt.subplot(2,2,2)
plt.plot(mean_fr_pyr_bla_s[100:-100], mean_sync_pyr_bla_s[100:-100], '.')
plt.title('pyr bla, Pearson coeff = '+str(corr_pyr_bla[1,0]))
plt.subplot(2,2,3)
plt.plot(mean_fr_int_hpc_s[100:-100], mean_sync_int_hpc_s[100:-100], '.')
plt.title('int hpc, Pearson coeff = '+str(corr_int_hpc[1,0]))
plt.subplot(2,2,4)
plt.plot(mean_fr_int_bla_s[100:-100], mean_sync_int_bla_s[100:-100], '.')
plt.title('int bla, Pearson coeff = '+str(corr_int_bla[1,0]))

Text(0.5, 1.0, 'int bla, Pearson coeff = -0.7551644915307446')

# Correlation between pourcent active neurons and mean firing rate

In [176]:
plt.figure()
g = sns.regplot(mean_fr_int_hpc_s.restrict(states['Rem']).values, pourcent_active_int_hpc_s.restrict(states['Rem']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_hpc_s.restrict(states['sws']).values, pourcent_active_int_hpc_s.restrict(states['sws']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_hpc_s[100:-100].restrict(states['wake']).values, pourcent_active_int_hpc_s[100:-100].restrict(states['wake']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'pourcent active neurons',
      title= f'Correlation between pourcent of active neurons and firing rate in hippocampal interneurons,  overall correlation coefficient = {corr[0,1]}')
g.legend(labels=['Rem', 'sws', 'Wake'])

C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [258]:
plt.figure()

bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_fr_pyr_bla_s.as_units('s'), color='g', label = 'pyr bla')
plt.plot(mean_fr_int_bla_s.as_units('s'), color='k', label='int bla')
plt.title('Mean firing rates. \n Session: Rat08-20130713')
plt.xlabel('Time (s)')
plt.legend()
plt.text(-1000,0.42, 'Time bins: 1s \nSmoothing parameters: \n    Number of points in window: 100 \n    Standard deviation: 50')
plt.figtext(0.12, 0.02, 'Methods: The mean firing rate was determined by computing the number of spikes for each neuron, in time bins of 1s, then by taking the mean over all neurons. \nThese two measures were then smoothed using a gaussian kernel across time (number of point in window = 100, standard deviation = 50)')

Text(0.12, 0.02, 'Methods: The mean firing rate was determined by computing the number of spikes for each neuron, in time bins of 1s, then by taking the mean over all neurons. \nThese two measures were then smoothed using a gaussian kernel across time (number of point in window = 100, standard deviation = 50)')

# Trash/Tests